# Library

In [1]:
import os, shutil

#必要なライブラリのインポート
import re, gc, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import warnings, random
import cv2

warnings.filterwarnings("ignore")

# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
import time
from sklearn.metrics import precision_recall_curve
# import optuna
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.cuda.amp import GradScaler

# import yaml
from tqdm import tqdm
import time
import copy
from collections import defaultdict

from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL
from visdom import Visdom
import timm

# Parameter

In [2]:
ARGS = {'DATA_DIR': 'D:\\テクノプロ・デザイン社 食品パッケージ画像解析チャレンジ',
  'OUT_DIR': './',
  'model_name': 'eva02_small_patch14_336',
  'n_fold': 5,
  'epochs': 100,
  'criterion': 'CrossEntropy',
  'image_size': (224, 224),
  'train_batch_size': 16,
  'test_batch_size': 16,
  'seed': 2023,
  'optimizer': 'AdamW',
  'learning_rate': 1e-05,
  'scheduler': 'CosineAnnealingLR',
  'min_lr': 1e-06,
  'T_max': 500,
  'n_accumulate': 1,
  'clip_grad_norm': 'None',
  'apex': True,
  'num_classes': 2,
  'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  }
ARGS

{'DATA_DIR': 'D:\\テクノプロ・デザイン社 食品パッケージ画像解析チャレンジ',
 'OUT_DIR': './',
 'model_name': 'eva02_small_patch14_336',
 'n_fold': 5,
 'epochs': 100,
 'criterion': 'CrossEntropy',
 'image_size': (224, 224),
 'train_batch_size': 16,
 'test_batch_size': 16,
 'seed': 2023,
 'optimizer': 'AdamW',
 'learning_rate': 1e-05,
 'scheduler': 'CosineAnnealingLR',
 'min_lr': 1e-06,
 'T_max': 500,
 'n_accumulate': 1,
 'clip_grad_norm': 'None',
 'apex': True,
 'num_classes': 2,
 'device': device(type='cuda', index=0)}

In [3]:
def get_logger(filename):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler2)
    return logger

#再現性を出すために必要な関数となります
def worker_init_fn(worker_id):
    torch.manual_seed(worker_id)
    random.seed(worker_id)
    np.random.seed(worker_id)
    torch.cuda.manual_seed(worker_id)
    os.environ['PYTHONHASHSEED'] = str(worker_id)

def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


LOGGER = get_logger(ARGS['OUT_DIR']+'train')
set_seed(ARGS["seed"])

# Create Folds

In [4]:
def create_folds(data, num_splits, seed):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
    labels = ["label"]
    data_labels = data[labels].values

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f

    return data

train = pd.read_csv(f"{ARGS['DATA_DIR']}/train.csv")
train = create_folds(train, num_splits=ARGS["n_fold"], seed=ARGS["seed"])
print("Folds created successfully")

train.head()

Folds created successfully


,image_name,label,kfold
0,0000.png,0,0
1,0001.png,1,3
2,0002.png,1,2
3,0003.png,1,1
4,0004.png,0,0


In [5]:
train["label"].value_counts()

label
1    1182
0     994
Name: count, dtype: int64

In [6]:
train['kfold'].value_counts()

kfold
0    436
3    435
2    435
1    435
4    435
Name: count, dtype: int64

# 前処理

In [7]:
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, df, transform, data_type):
        self.df = df
        self.data_type = data_type

        if self.data_type == "train":
            self.base_path = "D:\\テクノプロ・デザイン社 食品パッケージ画像解析チャレンジ\\train\\train"
            self.image_paths = df['image_name']
            self.labels = df['label']
        if self.data_type == "test":
            self.base_path = "D:\\テクノプロ・デザイン社 食品パッケージ画像解析チャレンジ\\test\\test"
            self.image_paths = df[0]

        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        image_path = self.image_paths[index]
        image_path = f"{self.base_path}\\{image_path}"

        image = Image.open(image_path)
        image = np.array(image.convert("RGB"))  # Convert PIL Image to numpy array

        if self.data_type == "train":
            label = self.labels[index]
            label = torch.tensor(label, dtype=torch.long)

            # Here use named argument 'image' for transform
            transformed = self.transform(image=image)
            return transformed['image'], label

        if self.data_type == "test":
            # Here use named argument 'image' for transform
            transformed = self.transform(image=image)
            return transformed['image']


In [8]:
import albumentations as albu
from albumentations.pytorch import transforms as AT

# Augumentation用
image_transform = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.RandomBrightnessContrast(p=0.3),
    albu.RandomGamma(gamma_limit=(85, 115), p=0.3),
    albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.10, rotate_limit=45, p=0.5),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])

# 学習用関数定義

In [9]:
def train_one_epoch(model, optimizer, train_loader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    running_score = []
    running_score_y = []
    scaler = GradScaler(enabled=ARGS["apex"])

    train_loss = []
    bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (images, targets) in bar:
      images = images.to(device)
      targets = targets.to(device)

      batch_size = targets.size(0)
      with torch.cuda.amp.autocast(enabled=ARGS["apex"]):
          outputs = model(images)
          loss = criterion(ARGS, outputs, targets)

      scaler.scale(loss).backward()

      if ARGS["clip_grad_norm"] != "None":
          grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), ARGS["clip_grad_norm"])
      else:
          grad_norm = None

      scaler.step(optimizer)
      scaler.update()

      optimizer.zero_grad()

      if scheduler is not None:
          scheduler.step()

      train_loss.append(loss.item())

      running_loss += (loss.item() * batch_size)
      dataset_size += batch_size

      epoch_loss = running_loss / dataset_size

      running_score.append(outputs.detach().cpu().numpy())
      running_score_y.append(targets.detach().cpu().numpy())

      score = get_score(running_score_y, running_score)

      bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                      Train_Acc=score[0],
                      Train_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr']
                      )
    gc.collect()
    return epoch_loss, score

In [10]:
@torch.no_grad()
def valid_one_epoch(args, model, optimizer, valid_loader, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    preds = []
    valid_targets = []
    softmax = nn.Softmax()

    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    for step, (images, targets) in enumerate(valid_loader):
      images = images.to(args["device"])
      targets = targets.to(args["device"])
      batch_size = targets.size(0)
      with torch.no_grad():
        outputs = model(images)
        predict = outputs.softmax(dim=1)
        loss = criterion(args, outputs, targets)

      running_loss += (loss.item() * batch_size)
      dataset_size += batch_size

      epoch_loss = running_loss / dataset_size

      preds.append(predict.detach().cpu().numpy())
      valid_targets.append(targets.detach().cpu().numpy())

      if len(set(np.concatenate(valid_targets))) == 1:
          continue
      score = get_score(valid_targets, preds)

      bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                      Valid_Acc=score[0],
                      Valid_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr'])

    return epoch_loss, preds, valid_targets, score

In [11]:
def one_fold(model, optimizer, schedulerr, device, num_epochs, fold):

    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_score = np.inf
    best_prediction = None

    early_stopping_rounds = 100 # **エポックの間改善がない場合、トレーニングを停止します。
    rounds_without_improvement = 0

    # Visdomオブジェクトの初期化
    viz = Visdom()

    best_score = -np.inf
    for epoch in range(1, 1+num_epochs):
      train_epoch_loss, train_score = train_one_epoch(model, optimizer,
                                              train_loader=train_loader,
                                              device=device, epoch=epoch)

      train_acc, train_auc = train_score

      val_epoch_loss, predictions, valid_targets, valid_score = valid_one_epoch(ARGS,
                                                                                model,
                                                                                optimizer,
                                                                                valid_loader,
                                                                                epoch=epoch)
      valid_acc, valid_auc = valid_score

      fold_legend = f'Fold {fold}'
        
        # データの更新
      viz.line(X=np.array([epoch]), Y=np.array([train_epoch_loss]), win='loss', name=f'Train Loss {fold_legend}', update='append', opts=dict(title='Loss', showlegend=True))
      viz.line(X=np.array([epoch]), Y=np.array([val_epoch_loss]), win='loss', name=f'Validation Loss {fold_legend}', update='append', opts=dict(title='Loss', showlegend=True))

      viz.line(X=np.array([epoch]), Y=np.array([train_auc]), win='auc', name=f'Train AUC {fold_legend}', update='append', opts=dict(title='AUC', showlegend=True))
      viz.line(X=np.array([epoch]), Y=np.array([valid_auc]), win='auc', name=f'Validation AUC {fold_legend}', update='append', opts=dict(title='AUC', showlegend=True))

      viz.line(X=np.array([epoch]), Y=np.array([train_acc]), win='acc', name=f'Train Accuracy {fold_legend}', update='append', opts=dict(title='Accuracy', showlegend=True))
      viz.line(X=np.array([epoch]), Y=np.array([valid_acc]), win='acc', name=f'Validation Accuracy {fold_legend}', update='append', opts=dict(title='Accuracy', showlegend=True))

        # .

      if valid_auc >= best_score:
        best_score = valid_auc
        rounds_without_improvement = 0

        print(f"{b_}Validation Score Improved ({best_epoch_score} ---> {valid_auc})")
        best_epoch_score = valid_auc
        best_model_wts = copy.deepcopy(model.state_dict())
        # PATH = f"Score-Fold-{fold}.bin"
        PATH = ARGS["OUT_DIR"] + f"Score-Fold-{fold}.bin"
        torch.save(model.state_dict(), PATH)
        # Save a model file from the current directory
        print(f"Model Saved{sr_}")

        best_prediction = np.concatenate(predictions, axis=0)[:,1]

      else:
        rounds_without_improvement += 1

      if rounds_without_improvement >= early_stopping_rounds:
            print(f"Early stopping after {early_stopping_rounds} epochs without improvement.")
            break  # トレーニングを早期に停止します。

    end = time.time()
    time_elapsed = end - start

    LOGGER.info('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    LOGGER.info("Best Score: {:.4f}".format(best_epoch_score))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_prediction, valid_targets

# モデルや損失関数、最適化関数等の定義

In [22]:
# model = models.efficientnet_v2_l()
# model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

: 

watasi no gamen wo mimasitane
sonna anataha kono seru ni mozi wo uttemitekudasai 
kitto tanosimemasu

kdlfjlkdgjldkfjgldjgsdjglsdgjjjjjlksdfjlkfjlakfaslkjfaksjfkasjfkjfjafjajffjasfljsajfkdasik

kghjkhkhkhkjhkj

In [13]:
def create_model(args):
    # Vit
    # model = models.vit_b_16(pretrained=True)
    # model.heads[0] = torch.nn.Linear(in_features=model.heads[0].in_features, out_features=args["num_classes"], bias=True)

    # Swin
    # model = models.swin_v2_t(pretrained=True)
    # model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=args["num_classes"], bias=True)

    # efficientnet
    # model = models.efficientnet_v2_l(pretrained=True)
    # model.classifier[-1] = torch.nn.Linear(in_features=model.classifier[-1].in_features, out_features=args["num_classes"], bias=True)

    # ConvNext
    # model = timm.create_model("convnextv2_tiny.fcmae", pretrained=True)
    # model = timm.create_model("convnext_base", pretrained=True)
    # model.head.fc = torch.nn.Linear(in_features=model.head.fc.in_features, out_features=args["num_classes"], bias=True)

    #Eva
    # model = timm.create_model('eva_large_patch14_336.in22k_ft_in22k_in1k', pretrained=True) #BEST
    model = timm.create_model('eva02_base_patch14_224', pretrained=True) #BEST
    model.reset_classifier(args["num_classes"])
    return model

def criterion(args, outputs, labels, class_weights=None):
    if args['criterion'] == 'CrossEntropy':
      return nn.CrossEntropyLoss(weight=class_weights).to(args["device"])(outputs, labels)
    elif args['criterion'] == "None":
        return None

def fetch_optimizer(optimizer_parameters, lr, betas, optimizer_name="Adam"):
    if optimizer_name == "Adam":
        optimizer = optim.Adam(optimizer_parameters, lr=lr)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(optimizer_parameters, lr=lr, betas=betas)
    return optimizer

def fetch_scheduler(args, train_size, optimizer):

    if args['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=args['T_max'],
                                                   eta_min=args['min_lr'])
    elif args['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=args['T_0'],
                                                             eta_min=args['min_lr'])
    elif args['scheduler'] == "None":
        scheduler = None

    return scheduler

def get_score(y_trues, y_preds):

    predict_list, targets_list = np.concatenate(y_preds, axis=0), np.concatenate(y_trues)
    predict_list_proba = predict_list.copy()[:, 1]
    predict_list = predict_list.argmax(axis=1)

    accuracy = accuracy_score(predict_list, targets_list)
    auc_score = roc_auc_score(targets_list, predict_list_proba)

    return (accuracy, auc_score)

def prepare_loaders(args, train, image_transform, fold):
    df_train = train[train.kfold != fold].reset_index(drop=True)
    df_valid = train[train.kfold == fold].reset_index(drop=True)

    train_dataset = CustomDataset(df_train, image_transform, data_type="train")
    valid_dataset = CustomDataset(df_valid, image_transform, data_type="train")

    train_loader = DataLoader(train_dataset, batch_size=args['train_batch_size'],
                              worker_init_fn=worker_init_fn(args["seed"]),
                              num_workers=0,
                              shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=args['test_batch_size'],
                              num_workers=0,
                              shuffle=False, pin_memory=True)

    return train_loader, valid_loader

# 学習

In [14]:
train_copy = train.copy()
LOGGER.info(ARGS)
for fold in range(0, ARGS['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    LOGGER.info(f"========== fold: {fold} training ==========")

    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(args=ARGS, train=train, image_transform=image_transform, fold=fold)

    model = create_model(ARGS)
    model = model.to(ARGS["device"])

    #損失関数・最適化関数の定義
    optimizer = fetch_optimizer(model.parameters(), optimizer_name=ARGS["optimizer"], lr=ARGS["learning_rate"], betas=(0.9, 0.999))

    scheduler = fetch_scheduler(args=ARGS, train_size=len(train_loader), optimizer=optimizer)

    model, predictions, targets = one_fold(model, optimizer, scheduler, device=ARGS["device"], num_epochs=ARGS["epochs"], fold=fold)

    train_copy.loc[train_copy[train_copy.kfold == fold].index, "oof"] = predictions

    del model, train_loader, valid_loader
    _ = gc.collect()
    torch.cuda.empty_cache()
    print()

scores = roc_auc_score(train_copy["label"].values, train_copy["oof"].values)
LOGGER.info(f"========== CV ==========")
LOGGER.info(f"CV: {scores:.4f}")

====== Fold: 0 ======


Setting up a new session...


[INFO] Using GPU: NVIDIA GeForce RTX 2080 Ti



  0%|          | 0/28 [00:05<?, ?it/s, Epoch=1, LR=9e-6, Valid_Acc=0.803, Valid_Auc=0.873, Valid_Loss=0.437]


Validation Score Improved (inf ---> 0.8730784725314334)
Model Saved


...

  0%|          | 0/28 [00:05<?, ?it/s, Epoch=77, LR=3.69e-6, Valid_Acc=0.936, Valid_Auc=0.976, Valid_Loss=0.338] 


Validation Score Improved (0.9753228487405702 ---> 0.9757810169202574)
Model Saved


  0%|          | 0/28 [00:05<?, ?it/s, Epoch=87, LR=1.93e-6, Valid_Acc=0.926, Valid_Auc=0.978, Valid_Loss=0.377]


Validation Score Improved (0.9757810169202574 ---> 0.9775604142692751)
Model Saved


  0%|          | 0/28 [00:05<?, ?it/s, Epoch=100, LR=6.89e-6, Valid_Acc=0.908, Valid_Auc=0.968, Valid_Loss=0.475]


In [15]:
# OOF
train_copy.to_csv(ARGS['OUT_DIR'] + f'oof.csv', index=False)

In [16]:
#sample_submit.csvを読み込みます
submit = pd.read_csv(f"{ARGS['DATA_DIR']}/sample_submit.csv", header=None)
submit.head()

,0,1
0,2176.png,0.042909
1,2177.png,0.912007
2,2178.png,0.511360
3,2179.png,0.127935
4,2180.png,0.119117


# Test

In [17]:
# test用のデータ拡張
image_transform_test = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2 ()
    ])
test_dataset = CustomDataset(submit, image_transform_test, data_type="test")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [18]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    predict_list = []

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, images in bar:
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images)
            #出力にソフトマックス関数を適用
            predicts = outputs.softmax(dim=1)

        predicts = predicts.cpu().detach().numpy()
        predict_list.append(predicts)
    predict_list = np.concatenate(predict_list, axis=0)
    #予測値が1である確率を提出します。
    predict_list = predict_list[:, 1]
    gc.collect()

    return predict_list

In [19]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = create_model(ARGS)
        model = model.to(device)

        #学習済みモデルの読み込み
        model.load_state_dict(torch.load(path))
        model.eval()

        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)

    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [20]:
MODEL_PATHS = [
    f"{ARGS['OUT_DIR']}/Score-Fold-{i}.bin" for i in range(ARGS["n_fold"])
]

predict_list = inference(MODEL_PATHS, test_loader, ARGS["device"])
submit[1] = predict_list
submit.head()

Getting predictions for model 1


100%|██████████| 69/69 [00:38<00:00,  1.78it/s]


Getting predictions for model 2


100%|██████████| 69/69 [00:26<00:00,  2.57it/s]


Getting predictions for model 3


...

Getting predictions for model 4


100%|██████████| 69/69 [00:26<00:00,  2.62it/s]


Getting predictions for model 5


100%|██████████| 69/69 [00:26<00:00,  2.60it/s]


,0,1
0,2176.png,0.064299
1,2177.png,0.798491
2,2178.png,0.000085
3,2179.png,0.999897
4,2180.png,0.000328


In [21]:
submit.to_csv(f"{ARGS['OUT_DIR']}/submission_CV{scores:.4f}.csv", index=False, header=None)